In [6]:
# pandas for reading and analyzing data
import pandas as pd
# numpy for numerical calcuations
import numpy as np
# os to find path of files 
import os

# tensorflow as machine learning library
import tensorflow as tf
# keras as open-source deep-learning library 
from tensorflow import keras
# building blocks of NN in Keras
from tensorflow.keras import layers
# earlyStop to stop training early
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

# IPython to Clear terminal output
import IPython
import IPython.display

# pickle to save dictionary in file
import pickle 
# time and timeit to provie a callback to logg model fitting time
import time
from timeit import default_timer as timer

# helper functions
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
cwd = os.path.normpath(os.getcwd()+ os.sep + os.pardir+ os.sep + os.pardir)
sys.path.insert(1, cwd + "/src/d00_utils") 
from federated_helper_functions import *
from model_helper_functions import *
from windowgenerator import *

In [12]:
def loadCompileEvaluateModel(path, window, MAX_EPOCHS):
    """
    load model, compile and evaluate on test window  
    
    :param: model path and window
    """           
    model = tf.keras.models.load_model(path)
    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[tf.keras.metrics.RootMeanSquaredError(), 
            tf.keras.metrics.MeanAbsolutePercentageError(),
            tf.keras.metrics.MeanAbsoluteError(),
        ]
    )
    
    #fit local model with client's data
    model.fit(
        window.train, 
        epochs=MAX_EPOCHS, 
        verbose=1, 
        validation_data=window.val,
        callbacks=[
            timetaken,
            #early_stopping, 
            #create_model_checkpoint(save_path=save_path), 
        ]
    )
    
    model_evaluation_test = model.evaluate(window.test)
    return model_evaluation_test

In [8]:
# Hyperparameter
OUT_STEPS = [12, 24] #Next 12 or 24 hours
INPUT_STEPS = 24
#Training epochs
MAX_EPOCHS = 2

#Data Analytics
print("Get data")
# get current working directory and go back one folder to main working directory
cwd = os.path.normpath(os.getcwd()+ os.sep + os.pardir+ os.sep + os.pardir)
#Read CSV file to pandas dataframe; encoding= 'unicode_escape': Decode from Latin-1 source code. Default UTF-8.
df = pd.read_csv(cwd+'/data/d03_data_processed/d03_data_processed.csv', encoding= 'unicode_escape', index_col='Date')
#Display smart meter names and amount
smart_meter_names = df.columns[2:-4]
print("Selected clients: ", len(smart_meter_names))

#Get clustered clients
N_CLUSTERS = 6
y = np.loadtxt(cwd+'/data/d04_clients_clustered/d04_clients_clustered.csv', delimiter=',').astype(int)
print("Clustered clients: ", y)

# Make Datasets for the 33 clients and for 5 and 7 features
ds_dict = makeDatasetsForclientsAndfeatures(smart_meter_names, df)
print("Created dictionary with datasets")

# Create Windows 
windows_dict = createDataWindows(y, smart_meter_names, INPUT_STEPS, OUT_STEPS, ds_dict, N_CLUSTERS)
print("Created Data windows")

windows_dict = {k: v for k, v in windows_dict.items() if k == 4}
print(windows_dict)

Get data
Selected clients:  33
Clustered clients:  [2 2 4 4 1 5 1 5 1 5 5 5 5 1 3 0 3 0 5 1 3 3 3 3 5 2 2 1 3 0 0 1 2]
Created dictionary with datasets
Created Data windows
{4: {'client_3_0141-ZE01-70': [Total window size: 36
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35]
Label column name(s): ['0141-ZE01-70'], Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label column name(s): ['0141-ZE01-70'], Total window size: 36
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35]
Label column name(s): ['0141-ZE01-70'], Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 

In [13]:
#Evaluate Results
forecasts_dict_LSTM_F5_H12 = {k: {} for k in range(N_CLUSTERS)}
forecasts_dict_CNN_F5_H12 = {k: {} for k in range(N_CLUSTERS)}
forecasts_dict_Transformer_F5_H12 = {k: {} for k in range(N_CLUSTERS)}

last_fedeated_round = 1

for idx, cluster in enumerate(windows_dict):
    
    #Get names of clients within cluster
    client_names = list()
    for client in windows_dict[cluster]:
        client_names.append(client)

    for i, client in enumerate(windows_dict[cluster].keys()):
        IPython.display.clear_output()
        print("Cluster----", cluster, "-----", client,"--------", i+1, "/", len(client_names))     
        
        #LSTM
        model_evaluation_test = loadCompileEvaluateModel(
            path = cwd + f"/data/d05_models/cluster{idx}/Federated_LSTM_F5_H12/FederatedRound{last_fedeated_round}",
            window = windows_dict[cluster][client][0], 
            MAX_EPOCHS = MAX_EPOCHS
        )
        #Save
        forecasts_dict_LSTM_F5_H12[cluster][client] = {
            'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
            'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1)) 
        }
        print("Saved LSTM")
        
        #CNN
        model_evaluation_test = loadCompileEvaluateModel(
            path = cwd + f"/data/d05_models/cluster{idx}/Federated_CNN_F5_H12/FederatedRound{last_fedeated_round}",
            window = windows_dict[cluster][client][0], 
            MAX_EPOCHS = MAX_EPOCHS
        )
        #Save
        forecasts_dict_CNN_F5_H12[cluster][client] = {
            'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
            'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1)) 
        }    
        print("Saved CNN")
        
        #Transformer
        model_evaluation_test = loadCompileEvaluateModel(
            path = cwd + f"/data/d05_models/cluster{idx}/Federated_Transformer_F5_H12/FederatedRound{last_fedeated_round}",
            window = windows_dict[cluster][client][0], 
            MAX_EPOCHS = MAX_EPOCHS
        )
        #Save
        forecasts_dict_Transformer_F5_H12[cluster][client] = {
            'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
            'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1)) 
        }
        print("Saved Transformer")

#Inintialize result dictionary
final_dict = InititalizeResultDictionary(learning_style="Federated")
final_dict['Federated']['LSTM']['H12']['F5'] = forecasts_dict_LSTM_F5_H12
final_dict['Federated']['CNN']['H12']['F5'] = forecasts_dict_CNN_F5_H12
final_dict['Federated']['Transformer']['H12']['F5'] = forecasts_dict_Transformer_F5_H12



Cluster---- 4 ----- client_4_0141-ZE01-74 -------- 2 / 2
Epoch 1/2
575/575 [==============================] - 77s 118ms/step - loss: 0.0602 - root_mean_squared_error: 0.2455 - mean_absolute_percentage_error: 65008864.0000 - mean_absolute_error: 0.1973 - val_loss: 0.0326 - val_root_mean_squared_error: 0.1806 - val_mean_absolute_percentage_error: 72346656.0000 - val_mean_absolute_error: 0.1542
Epoch 2/2
82/82 [==============================] - 3s 32ms/step - loss: 0.0341 - root_mean_squared_error: 0.1847 - mean_absolute_percentage_error: 79214488.0000 - mean_absolute_error: 0.1531
Saved LSTM
Epoch 1/2
575/575 [==============================] - 6s 7ms/step - loss: 0.0319 - root_mean_squared_error: 0.1787 - mean_absolute_percentage_error: 48392656.0000 - mean_absolute_error: 0.1319 - val_loss: 0.0315 - val_root_mean_squared_error: 0.1776 - val_mean_absolute_percentage_error: 73479328.0000 - val_mean_absolute_error: 0.1403
Epoch 2/2
82/82 [==============================] - 0s 2ms/step - los

NameError: name 'final_dict' is not defined

In [ ]:
with open(cwd + '/results/Federated_results_H12_F5.pkl', 'wb') as f:
    pickle.dump(final_dict, f)